In [1]:
import numpy as np
import pandas as pd

In [3]:
book = pd.read_csv('Books.csv')
user = pd.read_csv('Users.csv')
rating = pd.read_csv('Ratings.csv')

C:\Users\TOPPER\AppData\Local\Temp\ipykernel_11280\3651538461.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('Books.csv')


In [5]:
book.shape

(271360, 8)

In [6]:
user.shape

(278858, 3)

In [7]:
rating.shape

(1149780, 3)

In [9]:
book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [12]:
book.duplicated().sum()

0

In [13]:
rating.duplicated().sum()

0

In [14]:
user.duplicated().sum()

0

## Propularity Based Recommender System

In [59]:
rating_name = rating.merge(book,on='ISBN')

In [60]:
rating_name['Book-Rating'] = rating_name['Book-Rating'].astype(float)

In [61]:
rating_name['Book-Rating'].unique()

array([ 0.,  5.,  9.,  8.,  6.,  7.,  4., 10.,  3.,  2.,  1.])

In [62]:
num_rating_df = rating_name.groupby('Book-Title').count()['Book-Rating'].reset_index()

In [63]:
num_rating_df.rename(columns={'Book-Rating' : 'num_rating'},inplace=True)

In [64]:
num_rating_df

,Book-Title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [73]:
avg_rating_df = rating_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating' : 'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [74]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_rating,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [78]:
popular_df = popular_df[popular_df.num_rating>=250].sort_values('avg_rating',ascending=False).head(50)

In [83]:
popular_df = popular_df.merge(book,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-S','num_rating','avg_rating']]

## Collaborative recommender system


In [87]:
x = rating_name.groupby('User-ID')['Book-Rating'].count()>200
edu_user = x[x].index

In [93]:
filtered_rating = rating_name[rating_name['User-ID'].isin(edu_user)]

In [101]:
y = filtered_rating.groupby('Book-Title')['Book-Rating'].count()>=50
s_book = y[y].index

In [104]:
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(s_book)]

In [108]:
pt = final_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [110]:
pt.fillna(0,inplace=True)

In [111]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
from sklearn.metrics.pairwise import cosine_similarity

In [114]:
similarity_score = cosine_similarity(pt)

In [126]:
def recommend(book_name):
    index = np.where(pt.index==book_name)[0][0]
    similar_item = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:8]
    for i in similar_item:
        print(pt.index[i[0]])

In [127]:
recommend("The Handmaid's Tale")

Bastard Out of Carolina
A Civil Action
1984
The Kitchen God's Wife
The Red Tent (Bestselling Backlist)
The Virgin Suicides
Confessions of an Ugly Stepsister : A Novel
